# Create all_stations geojson
Eric Gagliano (egagli@uw.edu)   
February 7th, 2024

**Not meant to be run by the user!! This notebook documents the steps to pull in the SNOTEL and CCSS geojsons and do the csv downloads.**

In [1]:
#!mamba env create -q -f environment.yml --force
#!mamba activate snotel_ccss_env

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests
from urllib.parse import urlencode
import matplotlib.pyplot as plt
from datetime import datetime
from io import StringIO
import numpy as np
import mgrs
import datetime
import os
import time
import glob
import ee
ee.Initialize()
import sys
sys.path.append('snotel_ccss_stations')
import snotel_ccss_stations
import shapely
import ulmo
import fsspec
import tqdm

wsdl_url = 'https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'

In [3]:
%%time
snotel_stations_gdf = snotel_ccss_stations.all_snotel_stations(from_ulmo=True)

/home/jovyan/snotel_tools/snotel_tools/snotel_tools.py:61: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stations_df['endDate'] = pd.to_datetime(stations_df.endDate)
/home/jovyan/snotel_tools/snotel_tools/snotel_tools.py:62: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  stations_df['beginDate'] = pd.to_datetime(stations_df.beginDate)
/home/jovyan/snotel_tools/snotel_tools/snotel_tools.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stations_df['isActive'] = stations_df.isActive.

In [4]:
snotel_stations_gdf

,name,elevation_m,latitude,longitude,county,state,beginDate,endDate,HUC,isActive,mgrs,mountainRange,geometry
code,,,,,,,,,,,,,
301_CA_SNTL,Adin Mtn,1886.712036,41.235828,-120.791924,Modoc,California,1983-10-01 00:00:00,2100-01-01,180200021403,True,10TFL,Great Basin Ranges,POINT (-120.79192 41.23583)
907_UT_SNTL,Agua Canyon,2712.719971,37.522171,-112.271179,Kane,Utah,1994-10-01 00:00:00,2100-01-01,160300020301,True,12SUG,Colorado Plateau,POINT (-112.27118 37.52217)
916_MT_SNTL,Albro Lake,2529.840088,45.597229,-111.959023,Madison,Montana,1996-09-01 00:00:00,2100-01-01,100200050701,True,12TVR,Central Montana Rocky Mountains,POINT (-111.95902 45.59723)
1267_AK_SNTL,Alexander Lake,48.768002,61.749668,-150.889664,Matanuska-Susitna,Alaska,2014-08-28 00:00:00,2100-01-01,190205051106,True,05VPJ,NaN,POINT (-150.88966 61.74967)
908_WA_SNTL,Alpine Meadows,1066.800049,47.779572,-121.698471,King,Washington,1994-09-01 00:00:00,2100-01-01,171100100501,True,10TET,Cascade Range,POINT (-121.69847 47.77957)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
877_AZ_SNTL,Workman Creek,2103.120117,33.812420,-110.917732,Gila,Arizona,1938-12-01 00:00:00,2100-01-01,150601030802,True,12SWC,Southwest Basins and Ranges,POINT (-110.91773 33.81242)
1228_UT_SNTL,Wrigley Creek,2842.869629,39.132332,-111.356850,Sanpete,Utah,2012-10-01 00:00:00,2100-01-01,140600090303,True,12SVJ,Colorado Plateau,POINT (-111.35685 39.13233)
1197_UT_SNTL,Yankee Reservoir,2649.321533,37.747971,-112.774948,Iron,Utah,2012-10-01 00:00:00,2100-01-01,160300060202,True,12SUG,Colorado Plateau,POINT (-112.77495 37.74797)


In [5]:
%%time
ccss_stations_gdf = snotel_ccss_stations.all_ccss_stations()

In [8]:
ccss_stations_gdf

,name,elevation_m,latitude,longitude,county,state,HUC,mgrs,mountainRange,beginDate,endDate,geometry
code,,,,,,,,,,,,
CRM,Corral Meadow,2026.92,38.398000,-120.040000,Tuolumne,California,180400100302,10SGH,Sierra Nevada,NaT,NaT,POINT (-120.04000 38.39800)
CW2,Cottonwood Lakes 2,3383.28,36.483000,-118.217000,Inyo,California,180901030404,11SLA,Sierra Nevada,NaT,NaT,POINT (-118.21700 36.48300)
CUR,Courtright,2545.08,37.072000,-118.965000,Fresno,California,180300100602,11SLB,Sierra Nevada,NaT,NaT,POINT (-118.96500 37.07200)
DP2,Donner Park 2,1828.80,39.320000,-120.245000,Placer,California,160501020201,10SGJ,Sierra Nevada,NaT,NaT,POINT (-120.24500 39.32000)
FRW,Farewell Gap,2895.60,36.415211,-118.578979,Tulare,California,180300070202,11SLA,Sierra Nevada,2004-10-01,2024-02-08,POINT (-118.57898 36.41521)
...,...,...,...,...,...,...,...,...,...,...,...,...
SWJ,Swampy John,1676.40,41.397000,-122.993000,Siskiyou,California,180102080205,10TEL,Klamath Mountains,NaT,NaT,POINT (-122.99300 41.39700)
TMR,Tamarack Summit,2301.24,37.163750,-119.200531,Fresno,California,180400060903,11SLB,Sierra Nevada,2011-01-01,2024-02-08,POINT (-119.20053 37.16375)
GH3,Greyhorse Node 3,1889.76,39.122829,-120.361237,Placer,California,180201280203,10SGJ,Sierra Nevada,NaT,NaT,POINT (-120.36124 39.12283)


## Merge SNOTEL and CCSS stations

In [9]:
all_stations = pd.concat([snotel_stations_gdf,ccss_stations_gdf])
all_stations
condition = all_stations.index.str.contains("SNTL")
all_stations['network'] = np.where(condition,"SNOTEL","CCSS")
all_stations = all_stations[['name','network','elevation_m','latitude','longitude','state','HUC','mgrs','mountainRange','beginDate','endDate','geometry']]

In [10]:
all_stations

,name,network,elevation_m,latitude,longitude,state,HUC,mgrs,mountainRange,beginDate,endDate,geometry
code,,,,,,,,,,,,
301_CA_SNTL,Adin Mtn,SNOTEL,1886.712036,41.235828,-120.791924,California,180200021403,10TFL,Great Basin Ranges,1983-10-01,2100-01-01,POINT (-120.79192 41.23583)
907_UT_SNTL,Agua Canyon,SNOTEL,2712.719971,37.522171,-112.271179,Utah,160300020301,12SUG,Colorado Plateau,1994-10-01,2100-01-01,POINT (-112.27118 37.52217)
916_MT_SNTL,Albro Lake,SNOTEL,2529.840088,45.597229,-111.959023,Montana,100200050701,12TVR,Central Montana Rocky Mountains,1996-09-01,2100-01-01,POINT (-111.95902 45.59723)
1267_AK_SNTL,Alexander Lake,SNOTEL,48.768002,61.749668,-150.889664,Alaska,190205051106,05VPJ,NaN,2014-08-28,2100-01-01,POINT (-150.88966 61.74967)
908_WA_SNTL,Alpine Meadows,SNOTEL,1066.800049,47.779572,-121.698471,Washington,171100100501,10TET,Cascade Range,1994-09-01,2100-01-01,POINT (-121.69847 47.77957)
...,...,...,...,...,...,...,...,...,...,...,...,...
SWJ,Swampy John,CCSS,1676.400000,41.397000,-122.993000,California,180102080205,10TEL,Klamath Mountains,NaT,NaT,POINT (-122.99300 41.39700)
TMR,Tamarack Summit,CCSS,2301.240000,37.163750,-119.200531,California,180400060903,11SLB,Sierra Nevada,2011-01-01,2024-02-08,POINT (-119.20053 37.16375)
GH3,Greyhorse Node 3,CCSS,1889.760000,39.122829,-120.361237,California,180201280203,10SGJ,Sierra Nevada,NaT,NaT,POINT (-120.36124 39.12283)


In [12]:
all_stations.astype(dict(beginDate=str, endDate=str)).explore(column='network',cmap='bwr')

In [13]:
all_stations.to_file('all_stations.geojson')

## Download all SNOTEL and CCSS stations

In [ ]:
snotel_ccss_stations.download_snotel_data_csv(snotel_stations_gdf)

In [ ]:
snotel_ccss_stations.download_ccss_data_csv(ccss_stations_gdf)

## Add new column "data available"

In [ ]:
current_files = [os.path.splitext(os.path.basename(path))[0] for path in glob.glob('data/*.csv')]

In [ ]:
my_stations = stations_gdf[stations_gdf.index.isin(current_files)]